In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/딥러닝실습/archive.zip
!unzip /content/drive/MyDrive/ISIC2018_Task3_Test_Input.zip -d ISIC2018_Task3_Test_Input/
!unzip /content/drive/MyDrive/ISIC2018_Task3_Validation_Input.zip -d ISIC2018_Task3_Validation_Input/
#!unzip /content/drive/MyDrive/archive.zip
#!unzip /content/drive/MyDrive/normal.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ham10000_images_part_2/ISIC_0031036.jpg  
  inflating: ham10000_images_part_2/ISIC_0031037.jpg  
  inflating: ham10000_images_part_2/ISIC_0031038.jpg  
  inflating: ham10000_images_part_2/ISIC_0031039.jpg  
  inflating: ham10000_images_part_2/ISIC_0031040.jpg  
  inflating: ham10000_images_part_2/ISIC_0031041.jpg  
  inflating: ham10000_images_part_2/ISIC_0031042.jpg  
  inflating: ham10000_images_part_2/ISIC_0031043.jpg  
  inflating: ham10000_images_part_2/ISIC_0031044.jpg  
  inflating: ham10000_images_part_2/ISIC_0031045.jpg  
  inflating: ham10000_images_part_2/ISIC_0031046.jpg  
  inflating: ham10000_images_part_2/ISIC_0031047.jpg  
  inflating: ham10000_images_part_2/ISIC_0031048.jpg  
  inflating: ham10000_images_part_2/ISIC_0031049.jpg  
  inflating: ham10000_images_part_2/ISIC_0031050.jpg  
  inflating: ham10000_images_part_2/ISIC_0031051.jpg  
  inflating: ham10000_images_part_2/ISIC_0031052.jpg  
  inflating: ham10000_images_

In [3]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
import itertools
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from keras.callbacks import EarlyStopping

import keras_tuner as kt


In [5]:
test_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                    for x in glob(os.path.join('ISIC2018_Task3_Test_Input/', '*.jpg'))}

val_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                    for x in glob(os.path.join('ISIC2018_Task3_Validation_Input/', '*.jpg'))}
lesion_type_dict = {
    'nv': 0,
    'mel': 1,
    'bkl': 2,
    'bcc': 3,
    'akiec': 4,
    'vasc': 5,
    'df': 6
}


sex_dict={
    'male':1,'female':2,'unknown':3
}

test_df = pd.read_csv('/content/drive/MyDrive/ISIC2018_Task3_Test_GroundTruth.csv')


test_df['path'] = test_df['image'].map(test_path_dict.get)
test_df['cell_type_idx']=pd.Series(index=range(len(test_df)))
test_df['sex_int']=pd.Series(index=range(len(test_df)))
test_df['age']=pd.Series(index=range(len(test_df)))

for i in range(len(test_df)):
  diseases=test_df.iloc[i][1:8].to_numpy()
  test_df['cell_type_idx'][i]=lesion_type_dict[test_df.iloc[i][1:8].index[diseases.argmax()].lower()]
  test_df['sex_int'][i]=3
  test_df['age']=50



test_df['photo'] = test_df['path'].map(lambda x: np.asarray(Image.open(x).resize((60,45))))


x_test=np.empty((len(test_df['photo']),45,60,3))
for i in range(len(test_df['photo'])):
   x_test[i]=test_df['photo'][i]

x_test=x_test/255.0
y_test = test_df['cell_type_idx']
y_test = to_categorical(y_test, num_classes = 7)

val_df = pd.read_csv('/content/drive/MyDrive/ISIC2018_Task3_Validation_GroundTruth.csv')


val_df['path'] = val_df['image'].map(val_path_dict.get)
val_df['cell_type_idx']=pd.Series(index=range(len(val_df)))
val_df['sex_int']=pd.Series(index=range(len(val_df)))
val_df['age']=pd.Series(index=range(len(val_df)))

for i in range(len(val_df)):
  diseases=val_df.iloc[i][1:8].to_numpy()
  val_df['cell_type_idx'][i]=lesion_type_dict[val_df.iloc[i][1:8].index[diseases.argmax()].lower()]
  val_df['sex_int'][i]=3
  val_df['age']=50



val_df['photo'] = val_df['path'].map(lambda x: np.asarray(Image.open(x).resize((60,45))))


x_validate=np.empty((len(val_df['photo']),45,60,3))
for i in range(len(val_df['photo'])):
   x_validate[i]=val_df['photo'][i]

x_validate=x_validate/255.0
y_validate=val_df['cell_type_idx']
y_validate = to_categorical(y_validate, num_classes = 7)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-5-c85ea2c86e0b>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cell_type_idx'][i]=lesion_type_dict[test_df.iloc[i][1:8].index[diseases.argmax()].lower()]
<ipython-input-5-c85ea2c86e0b>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['sex_int'][i]=3
<ipython-input-5-c85ea2c86e0b>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cell_type_idx'][i]=lesion_t

In [6]:
# This dictionary is useful for displaying more human-friendly labels later on

imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                      for x in glob(os.path.join('ham10000*/', '*.jpg'))}

#normal_path=[x for x in glob(os.path.join('/content/Oily-Dry-Skin-Types/*/*','*.jpg'))]
# imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
#                     for x in glob(os.path.join('/content/drive/MyDrive/archive/*/', '*.jpg'))}

lesion_type_map = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}


In [7]:
skin_df = pd.read_csv('HAM10000_metadata.csv')
#skin_df = pd.read_csv('/content/drive/MyDrive/archive/HAM10000_metadata.csv')




In [8]:
skin_df['path'] = skin_df['image_id'].map(imageid_path_dict.get)
skin_df['cell_type'] = skin_df['dx'].map(lesion_type_dict.get)
skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes

In [9]:
#for i in normal_path:
#  skin_df.loc[len(skin_df),'path']=i

In [10]:
skin_df['age'].fillna((skin_df['age'].mean()),inplace=True)
#skin_df['cell_type_idx'].fillna(7, inplace=True)

In [11]:
skin_df['cell_type_idx'].value_counts()

cell_type_idx
0    6705
1    1113
2    1099
3     514
4     327
5     142
6     115
Name: count, dtype: int64

In [12]:
skin_df['image'] = skin_df['path'].map(lambda x:np.asarray(Image.open(x).resize((60,45))))

In [13]:
features=np.empty((len(skin_df['image']),45,60,3))
for i in range(len(skin_df['image'])):
   features[i]=skin_df['image'][i]
label=skin_df['cell_type_idx']

In [14]:
data_augmentation = tf.keras.Sequential([
  #layers.RandomFlip("horizontal"),
  layers.RandomRotation(factor=(-0.1,0.1)),
  layers.RandomTranslation(height_factor=(-0.1,0.1),width_factor=(-0.1,0.1))
])

In [15]:
aug_amount=skin_df["cell_type_idx"].value_counts()[0]+skin_df["cell_type_idx"].value_counts()[1]*3 + skin_df["cell_type_idx"].value_counts()[2]*3+skin_df["cell_type_idx"].value_counts()[3]*4+skin_df["cell_type_idx"].value_counts()[4]*5+skin_df["cell_type_idx"].value_counts()[5]*6+skin_df["cell_type_idx"].value_counts()[6]*6

aug_img_features=np.empty((aug_amount,45,60,3))
aug_label=np.empty(aug_amount)

In [16]:
aug_amount

18574

In [17]:
cnt=0
for i in range(len(label)):
  amplify=0
  if label[i]==1 or label[i]==2:
    amplify=3
    #6배
  elif label[i]==3:
    amplify=4
    #12배
  elif label[i]==4:
    amplify=5
    #20배
  elif label[i]==5 or label[i]==6:
    amplify=6
    #60배
  else:
    aug_img_features[cnt]=features[i]
    aug_label[cnt]=label[i]
    cnt+=1
    continue
  #print(f"{i+1}번 라벨 처리중")
  for j in range(amplify):
    aug_img_features[cnt]=data_augmentation(features[i])
    aug_label[cnt]=label[i]
    cnt+=1



In [18]:
idx=np.random.permutation(list(range(len(aug_label))))
aug_img_features=aug_img_features[idx]
aug_label=aug_label[idx]

In [24]:
x_train, y_train =aug_img_features, aug_label
x_train=x_train / 255.0  # Normalize pixel values to [0, 1]
y_train = to_categorical(y_train)  # One-hot encode labels

input_shape = x_train.shape[1:]
num_classes = y_train.shape[1]

# Define the CNN architecture
def create_cnn(hp):
    filter_1 = hp.Int("filter_1", min_value=16, max_value=512, step=16)
    filter_2 = hp.Int("filter_2", min_value=16, max_value=512, step=16)
    filter_3 = hp.Int("filter_3", min_value=16, max_value=512, step=16)
    filter_4 = hp.Int("filter_4", min_value=16, max_value=256, step=16)
    filter_5 = hp.Int("filter_5", min_value=16, max_value=512, step=16)
    filter_6 = hp.Int("filter_6", min_value=16, max_value=512, step=16)
    filter_7 = hp.Int("filter_7", min_value=16, max_value=512, step=16)
    filter_8 = hp.Int("filter_8", min_value=16, max_value=512, step=16)
    filter_9 = hp.Int("filter_9", min_value=16, max_value=512, step=16)
    filter_10 = hp.Int("filter_10", min_value=16, max_value=512, step=16)

    dropout1 = hp.Float("dropout1",min_value=0.2,max_value=0.5,step=0.05)
    dropout2 = hp.Float("dropout2",min_value=0.2,max_value=0.5,step=0.05)
    dropout3 = hp.Float("dropout3",min_value=0.2,max_value=0.5,step=0.05)
    dropout4 = hp.Float("dropout4",min_value=0.2,max_value=0.5,step=0.05)
    dropout5 = hp.Float("dropout5",min_value=0.2,max_value=0.5,step=0.05)
    dropout6 = hp.Float("dropout6",min_value=0.2,max_value=0.5,step=0.05)
    dropout7 = hp.Float("dropout7",min_value=0.2,max_value=0.5,step=0.05)
    dropout8 = hp.Float("dropout8",min_value=0.2,max_value=0.5,step=0.05)
    dropout9 = hp.Float("dropout9",min_value=0.2,max_value=0.5,step=0.05)
    dropout10 = hp.Float("dropout10",min_value=0.2,max_value=0.5,step=0.05)

    model = models.Sequential([
        layers.Input(shape=input_shape),
        #layers.Dropout(0.2),

        layers.Conv2D(filter_1, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(dropout1),
        #layers.BatchNormalization(),

        layers.Conv2D(filter_2, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(dropout2),
        #layers.BatchNormalization(),


        layers.Conv2D(filter_3, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(dropout3),
        # layers.BatchNormalization(),

        # layers.Conv2D(filter_4, (3, 3), activation='relu',padding='same'),
        # layers.MaxPooling2D((2, 2)),
        # layers.Dropout(dropout4),

        # layers.Conv2D(filter_5, (3, 3), activation='relu',padding='same'),
        # layers.MaxPooling2D((2, 2),padding='same'),
        # layers.Dropout(dropout5),

        layers.Flatten(),

        # layers.Dense(filter_7, activation='relu'),
        # layers.Dropout(dropout7),
        #layers.BatchNormalization(),

        # layers.Dense(filter_8, activation='relu'),
        # layers.Dropout(dropout8),

        layers.Dense(filter_9, activation='relu'),
        layers.Dropout(dropout9),


        layers.Dense(filter_10, activation='relu'),
        layers.Dropout(dropout10),

        layers.Dense(num_classes, activation='softmax')
    ])
    optimizer = hp.Choice("optimizer", values=["rmsprop", "adam"])
    lr = hp.Float("lr", min_value=1e-6, max_value=1e-3, sampling="log")
    if optimizer == "rmsprop":
        opt = keras.optimizers.RMSprop(learning_rate=lr)
    else:
        opt = keras.optimizers.Adam(learning_rate=lr)

    model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the CNN model


tuner = kt.RandomSearch(create_cnn, objective="val_accuracy",
							  max_trials=5, seed=42)


callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)]
tuner.search(x_train, y_train, batch_size=128,epochs=100,validation_data=(x_test, y_test), callbacks=callbacks, verbose=2)

best_models = tuner.get_best_models(1)
final_model = best_models[0]
# Train the model
#history = final_model.fit(datagen.flow(x_train, y_train, batch_size=128),epochs=100000, validation_data=(x_validate, y_validate),callbacks=EarlyStopping(patience=20))

# Evaluate the model
loss, accuracy = final_model.evaluate(x_test, y_test)
loss_v, accuracy_v = final_model.evaluate(x_validate, y_validate)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
final_model.save("only_cnn")

# Plot training history
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0, 1])
# plt.legend(loc='lower right')
# plt.show()

Trial 5 Complete [00h 08m 52s]
val_accuracy: 0.6613756418228149

Best val_accuracy So Far: 0.6719576716423035
Total elapsed time: 00h 34m 56s
7/7 [==============================] - 0s 29ms/step - loss: 0.5514 - accuracy: 0.8083
Validation: accuracy = 0.808290  ;  loss_v = 0.551435
Test: accuracy = 0.671958  ;  loss = 0.944797


In [25]:
final_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 43, 58, 416)       11648     
                                                                 
 max_pooling2d (MaxPooling2  (None, 21, 29, 416)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 21, 29, 416)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 27, 80)        299600    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 9, 13, 80)         0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 9, 13, 80)         0

In [27]:
import shutil
# 원본 경로
source_path = '/content/only_cnn'

# 목적지 경로
destination_path = '/content/drive/MyDrive/only_cnn13'

# 목적지 경로가 없으면 생성
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# '/content' 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.move(file_path, destination_path)

In [23]:
!rm -rf /content/untitled_project